# Inference of causal graphs from data

Author: Marcell Stippinger

Date: 2025-10-10

## Contents

* Generate and fit VAR and VMA models
* Analyze Granger causality

## Imports and plotting functions

In [ ]:
import numpy as np
from scipy import stats
from sklearn.utils import check_random_state
from typing import NamedTuple, Optional, Tuple, Any
#stationarity testing
from statsmodels.tsa.stattools import adfuller, kpss
#autocorrelation and partial autocorrelation
from statsmodels.tsa.stattools import acf, pacf
#VAR and VMA models
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.api import VARMAX
# Granger causality test
from statsmodels.tsa.stattools import grangercausalitytests
# Visualization
import matplotlib.pyplot as plt

In [ ]:
def plot_ts(X, E=None):
    """Plot time series data.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Time series data to plot.
    """
    n_samples, n_features = X.shape
    fig, axes = plt.subplots(n_features, 1, figsize=(6, 4), sharex=True)
    if n_features == 1:
        axes = [axes]
    for i in range(n_features):
        axes[i].axhline(0, color='gray', linestyle='--', linewidth=0.5)
        axes[i].plot(X[:, i])
        if E is not None:
            axes[i].plot(E[:, i], linestyle='None', marker='o', markersize=3, alpha=0.7, label='Noise')
        axes[i].set_title(f'Time Series {i+1}')
        axes[i].set_ylabel('Value')
    axes[-1].set_xlabel('Time')
    plt.tight_layout()
    plt.show()

## VAR(p) models

In [ ]:
def gen_var_by_hand(k: int, p: int, n: int, seed: int = None,
                    coeff_max: float = 0.5, coupling_mask: np.array = 1,
                    return_noise: bool = False):
    """
    Generate a VAR(p) process by hand.
    
    Parameters:
    k (int): Number of variables.
    p (int): Order of the VAR process.
    n (int): Length of the time series to generate.
    seed (int, optional): Random seed for reproducibility. Defaults to None.
    
    Returns:
    np.ndarray: Generated VAR(p) time series of shape (n, k).
    """
    rng = check_random_state(seed)
    burn_in = 100
    
    # Generate random coefficients for the VAR(p) model
    A = [coupling_mask * rng.uniform(-coeff_max, coeff_max, size=(k, k)) for _ in range(p)]
    
    # Initialize the time series with zeros
    Y = np.zeros((n + burn_in, k))  # Extra 100 for burn-in
    
    # Generate white noise
    E = rng.normal(size=(n + burn_in, k))
    
    # Generate the VAR(p) process
    for t in range(p, n + burn_in):
        Y[t] = sum(A[i] @ Y[t - i - 1] for i in range(p)) + E[t]
    
    if return_noise:
        return A, Y[burn_in:], E[burn_in:]  # Discard the burn-in period
    return A, Y[burn_in:]  # Discard the burn-in period

In [ ]:
ts_var_coeff, ts_var, ts_var_drive = gen_var_by_hand(2, 3, 200, coeff_max=0.5, seed=20251010, return_noise=True)
plot_ts(ts_var, ts_var_drive)

TODO:
- use a single time series
- explore the effect of model order $p$
- explore the effect of coeff_max on stationarity

## VMA(q) models

In [ ]:
def gen_vma_by_hand(k: int, q: int, n: int, seed=20251010,
                    coeff_min: float = -0.5, coeff_max: float = 0.5, coupling_mask: np.array = 1,
                    return_noise: bool = False):
    """
    Generate a VMA(q) process by hand.
    
    Parameters:
    k (int): Number of variables.
    q (int): Order of the VMA process.
    n (int): Length of the time series to generate.
    seed (int, optional): Random seed for reproducibility. Defaults to None.
    
    Returns:
    np.ndarray: Generated VMA(q) time series of shape (n, k).
    """
    rng = check_random_state(seed)
    burn_in = 100
    
    # Generate random coefficients for the VMA(q) model
    B = [coupling_mask * rng.uniform(coeff_min, coeff_max, size=(k, k)) for _ in range(q)]
    
    # Initialize the time series with zeros
    Y = np.zeros((n + burn_in, k))  # Extra 100 for burn-in
    
    # Generate white noise
    E = rng.normal(size=(n + burn_in, k))
    
    # Generate the VMA(q) process
    for t in range(q, n + burn_in):
        Y[t] = E[t] + sum(B[i] @ E[t - i - 1] for i in range(q))
    
    if return_noise:
        return B, Y[burn_in:], E[burn_in:]  # Discard the burn-in period
    return B, Y[burn_in:]  # Discard the burn-in period

In [ ]:
# Make all the coefficients positive and similar amplitude
ts_vma_coeff, ts_vma, ts_vma_drive = gen_vma_by_hand(1, 5, 200, coeff_min=0.90, coeff_max=0.99, seed=20251010, return_noise=True)
plot_ts(ts_vma, ts_vma_drive)

In [ ]:
ts_vma_coeff, ts_vma, ts_vma_drive = gen_vma_by_hand(2, 3, 200, coeff_max=0.5, seed=20251010, return_noise=True)
plot_ts(ts_vma, ts_vma_drive)

TODO:
- use a single time series
- explore the effect of model order p (tip: divide the vma time series by the model order for plotting)
- explore the effect of coeff_max on stationarity

## About stationarity tests

* ADF: null hypothesis: unit root (non-stationary), can conclude stationarity if $p$-value < alpha
* KPSS: null hypothesis: stationary, can conclude non-stationarity if $p$-value < alpha

In [ ]:
# help(adfuller)
# help(kpss)

In [ ]:
class ADFullerResult(NamedTuple):
    adf : float
    pvalue : float
    usedlag : int
    nobs : int
    critical_values : dict
    icbest : float
    resstore : Any = None

adf_result = ADFullerResult(*adfuller(ts_var[:, 0], maxlag=5))
adf_result

In [ ]:
class KPSSResult(NamedTuple):
    kpss_stat : float
    p_value : float
    lags : int
    crit : dict
    resstore : Any = None

kpss_result = KPSSResult(*kpss(ts_var[:, 1], nlags='auto'))
kpss_result

## Autocorrelograms

In [ ]:
def plot_autocorrelograms(x: np.ndarray, lags: int = 40):
    """Plot ACF and PACF of a time series.

    Parameters
    ----------
    x : array-like, shape (n_samples,)
        Time series data.
    lags : int
        Number of lags to include in the plots.
    """
    acf_vals = acf(x, nlags=lags)
    pacf_vals = pacf(x, nlags=lags)

    fig, axes = plt.subplots(2, 1, figsize=(6, 4))

    axes[0].stem(range(lags + 1), acf_vals)
    axes[0].set_title('Autocorrelation Function (ACF)')
    axes[0].set_xlabel('Lags')
    axes[0].set_ylabel('ACF')

    axes[1].stem(range(lags + 1), pacf_vals)
    axes[1].set_title('Partial Autocorrelation Function (PACF)')
    axes[1].set_xlabel('Lags')
    axes[1].set_ylabel('PACF')

    plt.tight_layout()
    plt.show()

In [ ]:
# AR model
coeff_ar, ts_ar = gen_var_by_hand(1, 5, 200, coeff_max=0.5, seed=20251010)
coeff_ar = np.array(coeff_ar)[:, 0, 0]
ts_ar = ts_ar[:, 0]
plot_autocorrelograms(ts_ar, lags=40)

In [ ]:

# MA model
coeff_ma, ts_ma = gen_vma_by_hand(1, 5, 200, coeff_max=1.0, seed=20251011)
coeff_ma = np.array(coeff_ma)[:, 0, 0]
ts_ma = ts_ma[:, 0]
plot_autocorrelograms(ts_ma, lags=40)

## Granger causality test

We can follow, for example

Ding, M., Chen, Y., & Bressler, S. L. (2006). Granger Causality: Basic Theory and Application to Neuroscience. February. https://doi.org/10.1002/9783527609970.ch17

In [ ]:
adjacency_matrix = np.array([[0, 1],
                             [0, 0]])

coupled_coeff, coupled_ts = gen_var_by_hand(2, 2, 200, coeff_max=0.5, seed=20251010, coupling_mask=adjacency_matrix.T)
plot_ts(coupled_ts)

In [ ]:
# Autocorrelograms of the two observed time series do not reveal the coupling
# It would be possible to create cross-correlograms as well
plot_autocorrelograms(coupled_ts[:, 0], lags=40)
plot_autocorrelograms(coupled_ts[:, 1], lags=40)

In [ ]:
# help(grangercausalitytests)  # 2nd column is tested to cause the 1st column

In [ ]:
# Y -> X
gr_yx = grangercausalitytests(coupled_ts, maxlag=4)

In [ ]:
# X -> Y
gr_xy = grangercausalitytests(coupled_ts[:, ::-1], maxlag=4)

Explain the results

- which tests are significant
- for what lag

## Explore some real-world data sets as well



### Epileptic EEG Dataset @ Mendeley Data

This dataset includes the EEG of 6 epileptic patients recorded at the Epilepsy monitoring unit of the American university of Beirut Medical Center between January 2014 and July 2015. The data represents measurements from 21 scalp electrodes, following the 10-20 electrode system, sampled at 500 Hz . All channels have been bandpass filtered between 1/1.6 Hz and 70Hz while filtering out the 50Hz (electrical utility frequency).  Some channels have been omitted from specific recordings due to artifact constraints. 

* By Wassim Nasreddine
* Published: 16 March 2021| Version 1 | DOI: 10.17632/5pc2j46cbc.1
* Find here: https://data.mendeley.com/datasets/5pc2j46cbc/1

In [ ]:
# Download the epileptic dataset
!mkdir data

import requests
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
epileptic = {
    'x_test': 'https://data.mendeley.com/public-files/datasets/5pc2j46cbc/files/93b81166-0e48-4dc0-ac20-b7167f7606c5/file_downloaded',
    'x_train': 'https://data.mendeley.com/public-files/datasets/5pc2j46cbc/files/169dca1c-4992-43d3-9c94-030de59c2524/file_downloaded',
    'y_test': 'https://data.mendeley.com/public-files/datasets/5pc2j46cbc/files/adf1c2fd-81ef-4f87-86cc-56d75bba8c31/file_downloaded',
    'y_train': 'https://data.mendeley.com/public-files/datasets/5pc2j46cbc/files/62accb90-a1b2-4b50-bde5-fbe6096f165f/file_downloaded'
}

headers = {
    "User-Agent": user_agent,
    "Referer": "https://data.mendeley.com/",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}
with requests.Session() as sess:
    sess.headers.update(headers)
    for key, url in epileptic.items():
        outpath = f"data/{key}.npz"
        print("Downloading", key, "->", outpath)
        try:
            r = sess.get(url, stream=True, timeout=30)
            if r.status_code == 200:
                with open(outpath, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                print("Saved", outpath)
            else:
                print("Failed", key, r.status_code, r.reason)
                print(" Response headers:", dict(r.headers))
        except requests.exceptions.RequestException as e:
            print("Error downloading", key, e)


### Kaggle Epileptic seizures dataset

The original dataset from the reference consists of 5 different folders, each with 100 files, with each file representing a single subject/person. Each file is a recording of brain activity for 23.6 seconds. The corresponding time-series is sampled into 4097 data points. Each data point is the value of the EEG recording at a different point in time. So we have total 500 individuals with each has 4097 data points for 23.5 seconds.

* https://www.kaggle.com/datasets/chaditya95/epileptic-seizures-dataset